This notebook cuts the bounding boxes delimited by the coordinates in the JSON file and store them as separate files. The code also creates a copy of the original image with the bounding boxes drawn on it.

### System Setup

In [1]:
# Libraries

import json
import os
import re
from PIL import Image, ImageDraw

In [2]:
# Variables

# Base directory
source_base_dir = "source"  # Base directory for source data

# JSON file name
json_file_name = "_annotations.coco.json"

### Prepare Data

In [3]:
# Function to process a single directory
def process_directory(source_dir):
    json_path = os.path.join(source_dir, json_file_name)
    output_json_path = json_path  # Save the updated JSON in the same directory

    # Read the original COCO JSON file
    with open(json_path, "r") as f:
        coco_data = json.load(f)

    # Get list of images in the source directory (long names)
    image_files = {img for img in os.listdir(source_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))}

    # Create a mapping dictionary long names -> short names
    name_mapping = {}

    for img_name in image_files:
        # Extract the main identifier from the short name (example: "image_00003")
        base_name_match = re.match(r"(image_\d+)", img_name)
        if base_name_match:
            base_name = base_name_match.group(1) + ".jpg"  # Desired short format
            name_mapping[img_name] = base_name

    # Rename files in the source directory
    for old_name, new_name in name_mapping.items():
        old_path = os.path.join(source_dir, old_name)
        new_path = os.path.join(source_dir, new_name)
        os.rename(old_path, new_path)

    # Filter images and update names in the JSON
    filtered_images = []
    for img in coco_data["images"]:
        if img["file_name"] in name_mapping:
            img["file_name"] = name_mapping[img["file_name"]]  # Replace with the short name
            filtered_images.append(img)

    # Get the IDs of the selected images
    selected_image_ids = {img["id"] for img in filtered_images}

    # Filter annotations related to the selected images
    filtered_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in selected_image_ids]

    # Create the new JSON with corrected names
    filtered_coco = {
        "images": filtered_images,
        "annotations": filtered_annotations,
        "categories": coco_data["categories"]  # Keep original categories
    }

    # Save the new JSON in the same directory
    with open(output_json_path, "w") as f:
        json.dump(filtered_coco, f, indent=4)

    print(f"✅ Files renamed and new JSON saved at: {output_json_path}")

In [4]:
# Process all subdirectories in the source base directory
for subdir in os.listdir(source_base_dir):
    source_dir = os.path.join(source_base_dir, subdir)
    if os.path.isdir(source_dir):
        process_directory(source_dir)

✅ Files renamed and new JSON saved at: source/valid/_annotations.coco.json
✅ Files renamed and new JSON saved at: source/test/_annotations.coco.json
✅ Files renamed and new JSON saved at: source/train/_annotations.coco.json


### Process Images

In [5]:
# Function to crop and save images
def crop_and_save(image_path, bbox, output_path):
    image = Image.open(image_path)
    left, upper, width, height = bbox
    right = left + width
    lower = upper + height
    cropped_image = image.crop((left, upper, right, lower))
    cropped_image.save(output_path)

# Function to draw bounding boxes on the image and save it
def draw_bounding_boxes(image_path, bboxes, output_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    for bbox in bboxes:
        left, upper, width, height = bbox
        right = left + width
        lower = upper + height
        draw.rectangle([left, upper, right, lower], outline="red", width=2)
    image.save(output_path)

# Function to process a single directory
def process_directory(source_dir):
    json_path = os.path.join(source_dir, json_file_name)
    processed_dir = os.path.join(source_dir, 'processed')
    os.makedirs(processed_dir, exist_ok=True)

    # Load annotations from the JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Create a dictionary to map image_id to file_name
    image_id_to_filename = {image['id']: image['file_name'] for image in data['images']}

    # Dictionary to group bounding boxes by image_id
    image_bboxes = {}

    # Iterate over annotations and group bounding boxes by image_id
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        bbox = annotation['bbox']
        if image_id not in image_bboxes:
            image_bboxes[image_id] = []
        image_bboxes[image_id].append(bbox)

    # Iterate over images and process them
    for image_id, bboxes in image_bboxes.items():
        image_filename = image_id_to_filename[image_id]
        image_path = os.path.join(source_dir, image_filename)
        
        # Save the image with bounding boxes drawn
        output_path_with_boxes = os.path.join(processed_dir, f"{os.path.splitext(image_filename)[0]}_with_boxes.jpg")
        draw_bounding_boxes(image_path, bboxes, output_path_with_boxes)
        
        # Crop and save each bounding box individually
        for i, bbox in enumerate(bboxes):
            output_path = os.path.join(processed_dir, f"{os.path.splitext(image_filename)[0]}_crop_{i}.jpg")
            crop_and_save(image_path, bbox, output_path)
        
        print(f"Processed {image_filename} with {len(bboxes)} bounding boxes")

In [6]:
# Process all subdirectories in the source base directory
for subdir in os.listdir(source_base_dir):
    source_dir = os.path.join(source_base_dir, subdir)
    if os.path.isdir(source_dir):
        process_directory(source_dir)

Processed image_00060.jpg with 13 bounding boxes
Processed image_00086.jpg with 6 bounding boxes
Processed image_00167.jpg with 4 bounding boxes
Processed image_00221.jpg with 22 bounding boxes
Processed image_00114.jpg with 7 bounding boxes
Processed image_00058.jpg with 4 bounding boxes
Processed image_00251.jpg with 6 bounding boxes
Processed image_00191.jpg with 34 bounding boxes
Processed image_00406.jpg with 19 bounding boxes
Processed image_00216.jpg with 6 bounding boxes
Processed image_00040.jpg with 3 bounding boxes
Processed image_00139.jpg with 8 bounding boxes
Processed image_00101.jpg with 4 bounding boxes
Processed image_00275.jpg with 49 bounding boxes
Processed image_00274.jpg with 3 bounding boxes
Processed image_00170.jpg with 5 bounding boxes
Processed image_00003.jpg with 83 bounding boxes
Processed image_00250.jpg with 8 bounding boxes
Processed image_00359.jpg with 3 bounding boxes
Processed image_00397.jpg with 4 bounding boxes
Processed image_00018.jpg with 1 b